TASK 1 : AIR QUALITY EXTRACTION

importing libraries

In [60]:
import gpxpy
import gpxpy.gpx
import pandas as pd
import pytz
from pytz import all_timezones

In [61]:
import xml.etree.ElementTree as ET
import csv
from datetime import datetime


# Function to parse custom time format
def parse_custom_time(time_str):
    dt = datetime.strptime(time_str, "%m/%d/%Y, %I:%M:%S %p")
    return dt.strftime("%Y-%m-%d"), dt.strftime("%H:%M:%S")

Data Extraction from gpx

In [62]:
gpx_file_path = "danielle GPX .GPX"
tree = ET.parse(gpx_file_path)
root = tree.getroot()

namespaces = {"default": "http://www.topografix.com/GPX/1/1"}

data = []
for trkpt in root.findall(".//default:trkpt", namespaces):
    lat = trkpt.get("lat")
    lon = trkpt.get("lon")
    ele = trkpt.find("default:ele", namespaces).text
    time = trkpt.find("default:time", namespaces).text
    date, time = parse_custom_time(time)
    data.append(
        {
            "latitude": lat,
            "longitude": lon,
            "altitude (m)": ele,
            "date": date,
            "time": time,
        }
    )

In [63]:
gpx_data = pd.DataFrame(data)
gpx_data.head()

,latitude,longitude,altitude (m),date,time
0,34.021526,-118.288752,90.12919518072158,2024-04-16,15:55:47
1,34.021566,-118.288890,91.4642141405493,2024-04-16,15:55:50
2,34.021470,-118.288567,92.40057468786836,2024-04-16,15:55:54
3,34.021553,-118.288315,91.63135995343328,2024-04-16,15:56:01
4,34.021452,-118.288464,91.29162885062397,2024-04-16,15:56:26


Import GPX Data to CSV

In [64]:
gpx_data.to_csv("danielle.csv", index=False)

https://gist.github.com/pianosnake/b4a45ef6bgpx_data2ffb2e1b44bbcca107298

Function to get boundary box from lat,long

In [65]:
import math

EARTH_CIR_METERS = 40075016.686
degreesPerMeter = 360 / EARTH_CIR_METERS


def toRadians(degrees):
    return degrees * math.pi / 180


def latLngToBounds(lat, lng, zoom, width, height):
    metersPerPixelEW = EARTH_CIR_METERS / math.pow(2, zoom + 8)
    metersPerPixelNS = (
        EARTH_CIR_METERS / math.pow(2, zoom + 8) * math.cos(toRadians(lat))
    )

    shiftMetersEW = width / 2 * metersPerPixelEW
    shiftMetersNS = height / 2 * metersPerPixelNS

    shiftDegreesEW = shiftMetersEW * degreesPerMeter
    shiftDegreesNS = shiftMetersNS * degreesPerMeter

    minX = lng - shiftDegreesEW
    minY = lat - shiftDegreesNS
    maxX = lng + shiftDegreesEW
    maxY = lat + shiftDegreesNS

    return f"{minX:.4f},{minY:.6f},{maxX:.4f},{maxY:.6f}"

In [66]:
gpx_data["latitude"] = gpx_data["latitude"].astype(float)
gpx_data["longitude"] = gpx_data["longitude"].astype(float)

gpx_data["bbox"] = gpx_data.apply(
    lambda x: latLngToBounds(x["latitude"], x["longitude"], 12, 400, 400), axis=1
)

In [67]:
gpx_data.head()

,latitude,longitude,altitude (m),date,time,bbox
0,34.021526,-118.288752,90.12919518072158,2024-04-16,15:55:47,"-118.3574,33.964615,-118.2201,34.078437"
1,34.021566,-118.288890,91.4642141405493,2024-04-16,15:55:50,"-118.3576,33.964655,-118.2202,34.078477"
2,34.021470,-118.288567,92.40057468786836,2024-04-16,15:55:54,"-118.3572,33.964559,-118.2199,34.078381"
3,34.021553,-118.288315,91.63135995343328,2024-04-16,15:56:01,"-118.3570,33.964642,-118.2197,34.078464"
4,34.021452,-118.288464,91.29162885062397,2024-04-16,15:56:26,"-118.3571,33.964541,-118.2198,34.078363"


In [68]:
import requests
import csv
from io import StringIO


def get_air_quality(gpx_data):
    hour = gpx_data["time"].split(":")[0]
    header_array = [
        "Latitude",
        "Longitude",
        "UTC",
        "Parameter",
        "Unit",
        "AQI",
        "Category",
    ]
    url = f"https://www.airnowapi.org/aq/data/?startDate={gpx_data['date']}T{hour}&endDate={gpx_data['date']}T{hour}&parameters=PM25,PM10&BBOX={gpx_data['bbox']}&dataType=A&format=text/csv&verbose=0&monitorType=0&includerawconcentrations=0&API_KEY=342FB14E-3637-470D-BEAE-A5DF1E193ADB"
    response = requests.get(url)
    return response.text

In [295]:
import pandas as pd
from io import StringIO

results = []
for index, row in gpx_data.head(400).iterrows():
    results.append(get_air_quality(row))

results_str = "\n".join(results)

data_io = StringIO(results_str)
df_temp = pd.read_csv(data_io, header=None)

df_temp.columns = [
    "Latitude",
    "Longitude",
    "UTC",
    "Parameter",
    "AQI",
    "Category",
]


df_temp.head()
df_temp.to_csv("air_quality.csv", index=False)

In [80]:
for index, row in gpx_data.iloc[700:1000].iterrows():
    results.append(get_air_quality(row))

results_str = "\n".join(results)

data_io = StringIO(results_str)
df_temp = pd.read_csv(data_io, header=None)
df_temp = pd.concat(results)
df_temp.columns = [
    "Latitude",
    "Longitude",
    "UTC",
    "Parameter",
    "AQI",
    "Category",
]

with open("air_quality.csv", "a") as f:
    df_temp.to_csv(f, header=False, index=False)

KeyboardInterrupt: 

In [69]:
# ping2
# 34.024818	-118.278297
# 34.024978	-118.279339
# 34.026068	-118.277422
# 34.024614	-118.278548
# 34.021216	-118.287226
# 34.021744	-118.282471

# 08/04/24 10:00:00
# 10/04/24 10:00:00
# 11/04/24 10:00:00
# 15/04/24 10:00:00
# 16/04/24 10:00:00
# 20/04/24 10:00:00

# ping3
# 34.027001	-118.278625
# 34.026218	-118.277408
# 34.027072	-118.278668
# 33.991122	-118.467379
# 34.022084	-118.288444
# 34.021743	-118.281996

# 08/04/24 12:00:00
# 09/04/24 12:00:00
# 10/04/24 12:00:00
# 12/04/24 12:00:00
# 19/04/24 12:00:00
# 20/04/24 12:00:00

# ping4
# 34.02506	-118.279307
# 34.022252	-118.282362
# 34.021907	-118.280967
# 34.102116	-118.396932
# 34.021386	-118.288725
# 34.025186	-118.279007
# 34.024986	-118.27913


# 08/04/24 14:00:00
# 09/04/24 14:00:00
# 10/04/24 14:00:00
# 12/04/24 14:00:00
# 16/04/24 14:00:00
# 19/04/24 14:00:00
# 20/04/24 14:00:00

# ping5
# 34.022021	-118.284436
# 34.024202	-118.283437
# 34.021966	-118.283216
# 32.774334	-117.186616
# 34.026959	-118.278633
# 34.02222	-118.284611
# 34.022183	-118.284469
# 34.025143	-118.279262


# 09/04/24 16:00:00
# 10/04/24 16:00:00
# 11/04/24 16:00:00
# 13/04/24 16:00:00
# 15/04/24 16:00:00
# 16/04/24 16:00:00
# 18/04/24 16:00:00
# 20/04/24 16:00:00

# ping6
# 34.025203	-118.279196
# 32.706276	-117.157111
# 32.774302	-117.186542
# 34.025001	-118.279192
# 34.022143	-118.284522


# 12/04/24 18:00:00
# 13/04/24 18:00:00
# 14/04/24 18:00:00
# 15/04/24 18:00:00
# 18/04/24 18:00:00


sample_data = [
    {
        "latitude": 34.024818,
        "longitude": -118.278297,
        "date": "2024-04-08",
        "time": "10:00:00",
    },
    {
        "latitude": 34.024978,
        "longitude": -118.279339,
        "date": "2024-04-10",
        "time": "10:00:00",
    },
    {
        "latitude": 34.026068,
        "longitude": -118.277422,
        "date": "2024-04-11",
        "time": "10:00:00",
    },
    {
        "latitude": 34.024614,
        "longitude": -118.278548,
        "date": "2024-04-15",
        "time": "10:00:00",
    },
    {
        "latitude": 34.021216,
        "longitude": -118.287226,
        "date": "2024-04-16", 
        "time": "10:00:00",
    },
    {
        "latitude": 34.021744,
        "longitude": -118.282471,
        "date": "2024-04-20",
        "time": "10:00:00",
    },
    {
        "latitude": 34.027001,
        "longitude": -118.278625,
        "date": "2024-04-08",
        "time": "12:00:00",
    },
    {
        "latitude": 34.026218,
        "longitude": -118.277408,
        "date": "2024-04-09",
        "time": "12:00:00",
    },
    {
        "latitude": 34.027072,
        "longitude": -118.278668,
        "date": "2024-04-10",
        "time": "12:00:00",
    },
    {
        "latitude": 33.991122,
        "longitude": -118.467379,
        "date": "2024-04-12",
        "time": "12:00:00",
    },
    {
        "latitude": 34.022084,
        "longitude": -118.288444,
        "date": "2024-04-19",
        "time": "12:00:00",
    },
    {
        "latitude": 34.021743,
        "longitude": -118.281996,
        "date": "2024-04-20",
        "time": "12:00:00",
    },
    {
        "latitude": 34.02506,
        "longitude": -118.279307,
        "date": "2024-04-08",
        "time": "14:00:00",
    },
    {
        "latitude": 34.022252,
        "longitude": -118.282362,
        "date": "2024-04-09",
        "time": "14:00:00",
    },
    {
        "latitude": 34.021907,
        "longitude": -118.280967,
        "date": "2024-04-10",
        "time": "14:00:00",
    },
    {
        "latitude": 34.102116,
        "longitude": -118.396932,
        "date": "2024-04-12",
        "time": "14:00:00",
    },
    {
        "latitude": 34.021386,
        "longitude": -118.288725,
        "date": "2024-04-16",
        "time": "14:00:00",
    },
    {
        "latitude": 34.025186,
        "longitude": -118.279007,
        "date": "2024-04-19",
        "time": "14:00:00",
    },
    {
        "latitude": 34.024986,
        "longitude": -118.279130,
        "date": "2024-04-20",
        "time": "14:00:00",
    },
    {
        "latitude": 34.022021,
        "longitude": -118.284436,
        "date": "2024-04-09",
        "time": "16:00:00",
    },
    {
        "latitude": 34.024202,
        "longitude": -118.283437,
        "date": "2024-04-10",
        "time": "16:00:00",
    },
    {
        "latitude": 34.021966,
        "longitude": -118.283216,
        "date": "2024-04-11",
        "time": "16:00:00",
    },
    {
        "latitude": 32.774334,
        "longitude": -117.186616,
        "date": "2024-04-13",
        "time": "16:00:00",
    },
    {
        "latitude": 34.026959,
        "longitude": -118.278633,
        "date": "2024-04-15",
        "time": "16:00:00",
    },
    {
        "latitude": 34.02222,
        "longitude": -118.284611,
        "date": "2024-04-16",
        "time": "16:00:00",
    },
    {
        "latitude": 34.022183,
        "longitude": -118.284469,
        "date": "2024-04-18",
        "time": "16:00:00",
    },
    {
        "latitude": 34.025143,
        "longitude": -118.279262,
        "date": "2024-04-20",
        "time": "16:00:00",
    },
    {
        "latitude": 34.025203,
        "longitude": -118.279196,
        "date": "2024-04-12",
        "time": "18:00:00",
    },
    {
        "latitude": 32.706276,
        "longitude": -117.157111,
        "date": "2024-04-13",
        "time": "18:00:00",
    },
    {
        "latitude": 32.774302,
        "longitude": -117.186542,
        "date": "2024-04-14",
        "time": "18:00:00",
    },
    {
        "latitude": 34.025001,
        "longitude": -118.279192,
        "date": "2024-04-15",
        "time": "18:00:00",
    },
    {
        "latitude": 34.022143,
        "longitude": -118.284522,
        "date": "2024-04-18",
        "time": "18:00:00",
    },
]
# sample_data = [
#     {
#         "latitude": 34.019451,
#         "longitude": -118.289409,
#         "date": "2024-04-08",
#         "time": "8:00:00",
#     },
#     {
#         "latitude": 34.024496,
#         "longitude": -118.278309,
#         "date": "2024-04-11",
#         "time": "8:00:00",
#     },
#     {
#         "latitude": 34.025839,
#         "longitude": -118.27746,
#         "date": "2024-04-13",
#         "time": "8:00:00",
#     },
#     {
#         "latitude": 34.019611,
#         "longitude": -118.289287,
#         "date": "2024-04-15",
#         "time": "8:00:00",
#     },
#     {
#         "latitude": 34.01943,
#         "longitude": -118.289417,
#         "date": "2024-04-17",
#         "time": "8:00:00",
#     },
#     {
#         "latitude": 34.024762,
#         "longitude": -118.279302,
#         "date": "2024-04-19",
#         "time": "8:00:00",
#     },
# ]


sample_data_df = pd.DataFrame(sample_data)
sample_data_df["bbox"] = sample_data_df.apply(
    lambda x: latLngToBounds(x["latitude"], x["longitude"], 12, 400, 400), axis=1
)

sample_data_df.head()

results = []
for index, row in sample_data_df.iterrows():
    results.append(get_air_quality(row))


results_str = "\n".join(results)

data_io = StringIO(results_str)

df_temp = pd.read_csv(data_io, header=None)

df_temp.columns = [
    "Latitude",
    "Longitude",
    "UTC",
    "Parameter",
    "AQI",
    "Category",
]

df_temp.head()

with open("air_quality.csv", "a") as f:
    df_temp.to_csv(f, header=False, index=False)

data extraction from survey data

In [70]:
import pandas as pd
import os

directory = "surveyData"

dataframes = {}

for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        dataframes[filename] = pd.read_csv(os.path.join(directory, filename))

In [71]:
dataframes.keys()

dict_keys(['Ping1.csv', 'Ping2.csv', 'Ping3.csv', 'Ping6.csv', 'Ping4.csv', 'Ping5.csv'])

In [72]:
ping1 = dataframes["Ping1.csv"]
ping2 = dataframes["Ping2.csv"]
ping3 = dataframes["Ping3.csv"]
ping4 = dataframes["Ping4.csv"]
ping5 = dataframes["Ping5.csv"]
ping6 = dataframes["Ping6.csv"]

In [73]:
ping1.head()

,mbl_cod,rsp_id,instance_id,scheduled_start_local,timezone_offset,actual_start_local,SLEEP_9379186,HAPPY_JOYFUL_9379200,ENERGETIC_9379201,IRRITABLE_ANGRY_9379202,...,RESPIRATION,BODY_BATTERY,STEPS,CALORIES,FLOORS,INTENSITY_MINUTES,LONGITUDE,LATITUDE,AVG_AMP,VOX_ACTV
0,27181246,50695,1712592000,2024-04-08 09:00:00,-420,2024-04-08 09:34:46,3,7.0,6.0,6.0,...,0.0,0.0,0,0,0,0,0.000000,0.000000,0.0,0.0
1,27181246,50695,1712678400,2024-04-09 09:00:00,-420,2024-04-09 09:06:19,3,7.0,6.0,5.0,...,0.0,0.0,0,0,0,0,0.000000,0.000000,0.0,0.0
2,27181246,50695,1712764800,2024-04-10 09:00:00,-420,2024-04-10 09:33:15,4,7.0,6.0,5.0,...,0.0,0.0,0,0,0,0,34.019451,-118.289409,0.0,0.0
3,27181246,50695,1712851200,2024-04-11 09:00:00,-420,2024-04-11 09:00:05,3,1.0,2.0,3.0,...,0.0,0.0,0,0,0,0,34.024496,-118.278309,0.0,0.0
4,27181246,50695,1712937600,2024-04-12 09:00:00,-420,2024-04-12 09:04:59,3,NaN,NaN,NaN,...,0.0,0.0,0,0,0,0,0.000000,0.000000,0.0,0.0


In [74]:
from datetime import datetime, timedelta

ping1["bbox"] = ping1.apply(
    lambda x: latLngToBounds(x["LONGITUDE"], x["LATITUDE"], 12, 400, 400), axis=1
)


def isInsideBbox(lat, lng, bbox):
    lng = float(lng)
    lat = float(lat)
    bbox = bbox.split(",")
    if (
        lng >= float(bbox[0])
        and lng <= float(bbox[2])
        and lat >= float(bbox[1])
        and lat <= float(bbox[3])
    ):
        return True
    return False


results = []


air_quality = pd.read_csv("air_quality.csv")
ping1["actual_start_local"] = pd.to_datetime(ping1["actual_start_local"])
ping1["hour"] = ping1["actual_start_local"].dt.hour - 1
air_quality["time"] = pd.to_datetime(air_quality["UTC"])
air_quality["hour"] = air_quality["time"].dt.hour

ping1["date"] = ping1["actual_start_local"].dt.date

air_quality["date"] = air_quality["time"].dt.date

for index, row in ping1.iterrows():
    op = []
    for index1, row1 in air_quality.iterrows():
        if (
            isInsideBbox(row1["Latitude"], row1["Longitude"], row["bbox"])
            and row["hour"] == row1["hour"]
            and row["date"] == row1["date"]
        ):
            op.append(
                {
                    "Latitude": row1["Latitude"],
                    "Longitude": row1["Longitude"],
                    "UTC": row1["UTC"],
                    "Parameter": row1["Parameter"],
                    "AQI": row1["AQI"],
                    "Category": row1["Category"],
                }
            )
    results.append(pd.DataFrame(op))

In [75]:
avg_PM25 = {}
avg_PM10 = {}

for i, df in enumerate(results):
    if not df.empty:
        avg_PM25[i] = df[df["Parameter"] == "PM2.5"]["AQI"].mean()
        avg_PM10[i] = df[df["Parameter"] == "PM10"]["AQI"].mean()
    else:
        avg_PM25[i] = None
        avg_PM10[i] = None

ping1["avg_PM25"] = pd.Series(avg_PM25)
ping1["avg_PM10"] = pd.Series(avg_PM10)

ping1.to_csv("ping1_avgAQI.csv", index=False)

In [76]:
import pandas as pd
from datetime import datetime, timedelta


def process_ping_data(ping, air_quality_path="air_quality.csv"):
    ping["bbox"] = ping.apply(
        lambda x: latLngToBounds(x["LONGITUDE"], x["LATITUDE"], 12, 400, 400), axis=1
    )

    def isInsideBbox(lat, lng, bbox):
        lng = float(lng)
        lat = float(lat)
        bbox = bbox.split(",")
        if (
            lng >= float(bbox[0])
            and lng <= float(bbox[2])
            and lat >= float(bbox[1])
            and lat <= float(bbox[3])
        ):
            return True
        return False

    results = []

    air_quality = pd.read_csv(air_quality_path)

    ping["actual_start_local"] = pd.to_datetime(ping["actual_start_local"])
    ping["hour"] = ping["actual_start_local"].dt.hour - 1
    air_quality["time"] = pd.to_datetime(air_quality["UTC"])
    air_quality["hour"] = air_quality["time"].dt.hour

    ping["date"] = ping["actual_start_local"].dt.date

    air_quality["date"] = air_quality["time"].dt.date

    for index, row in ping.iterrows():
        op = []
        for index1, row1 in air_quality.iterrows():
            if (
                isInsideBbox(row1["Latitude"], row1["Longitude"], row["bbox"])
                and row["hour"] == row1["hour"]
                and row["date"] == row1["date"]
            ):
                op.append(
                    {
                        "Latitude": row1["Latitude"],
                        "Longitude": row1["Longitude"],
                        "UTC": row1["UTC"],
                        "Parameter": row1["Parameter"],
                        "AQI": row1["AQI"],
                        "Category": row1["Category"],
                    }
                )
        results.append(pd.DataFrame(op))

    avg_PM25 = {}
    avg_PM10 = {}

    for i, df in enumerate(results):
        if not df.empty:
            avg_PM25[i] = df[df["Parameter"] == "PM2.5"]["AQI"].mean()
            avg_PM10[i] = df[df["Parameter"] == "PM10"]["AQI"].mean()
        else:
            avg_PM25[i] = None
            avg_PM10[i] = None

    ping["avg_PM25"] = pd.Series(avg_PM25)
    ping["avg_PM10"] = pd.Series(avg_PM10)

    return ping

In [78]:
for key in dataframes.keys():
    dataframes[key] = process_ping_data(dataframes[key])

for key, df in dataframes.items():
    df.to_csv(f"./OutputData/{key}_avgAQI.csv", index=False)